<a href="https://colab.research.google.com/github/eunice-hong1/DS340_Group6_AnomalyDetectioninEmail/blob/main/Capstone_Project_Testing_Supervised%2C_Phishing_Ham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup**

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import metrics

#from sklearn.metrics import plot_roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.decomposition import PCA
from matplotlib.pyplot import figure
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit

import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/eunice-hong1/DS340_Group6_AnomalyDetectioninEmail/main/preprocessed_spam_ham_phishing.csv")

In [4]:
df.columns

Index(['hops', 'missing_subject', 'missing_to', 'missing_content-type',
       'missing_mime-version', 'missing_x-mailer',
       'missing_content-transfer-encoding', 'missing_x-mimeole',
       'missing_x-priority', 'missing_list-id', 'missing_lines',
       'missing_x-virus-scanned', 'missing_status', 'missing_content-length',
       'missing_precedence', 'missing_delivered-to',
       'missing_list-unsubscribe', 'missing_list-subscribe',
       'missing_list-post', 'missing_list-help', 'missing_x-msmail-priority',
       'missing_x-spam-status', 'missing_sender', 'missing_errors-to',
       'missing_x-beenthere', 'missing_list-archive', 'missing_reply-to',
       'missing_x-mailman-version', 'missing_x-miltered', 'missing_x-uuid',
       'missing_x-virus-status', 'missing_x-spam-level',
       'missing_x-spam-checker-version', 'missing_references',
       'missing_in-reply-to', 'missing_user-agent', 'missing_thread-index',
       'missing_cc', 'missing_received-spf', 'missing_x-orig

**Remove spam emails, only consider ham and phishing:**

In [5]:
df = df[df['label'] != 1]
print(df.shape)

(26508, 95)


In [6]:
df['label'].value_counts()

0    25220
2     1288
Name: label, dtype: int64

**Randomly Sample 1288 Ham emails to create a balanced dataset:**

In [7]:
df_ham = df[df['label'] == 0].sample(1288)
df_phish = df[df['label'] == 2]

In [8]:
df_phish = df_phish.assign(label=1)

In [9]:
df_new = df_ham._append(df_phish, ignore_index=True)
df_new

,hops,missing_subject,missing_to,missing_content-type,missing_mime-version,missing_x-mailer,missing_content-transfer-encoding,missing_x-mimeole,missing_x-priority,missing_list-id,...,domain_match_errors-to_reply-to,domain_match_sender_from,domain_match_references_reply-to,domain_match_references_in-reply-to,domain_match_references_to,domain_match_from_reply-to,domain_match_to_from,domain_match_to_message-id,domain_match_to_received,label
0,1,0,0,0,1,1,0,1,1,0,...,1,1,0,0,0,1,1,1,0,0
1,1,0,0,0,1,1,0,1,1,0,...,1,1,0,0,0,1,1,1,0,0
2,1,0,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,1,0,0,1,0
4,2,0,0,0,0,1,1,1,1,0,...,0,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571,0,0,0,0,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2572,1,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,1,1,0,1
2573,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2574,1,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [10]:
df_new = df_new.sample(frac=1)
df = df_new.reset_index(drop=True)
df

,hops,missing_subject,missing_to,missing_content-type,missing_mime-version,missing_x-mailer,missing_content-transfer-encoding,missing_x-mimeole,missing_x-priority,missing_list-id,...,domain_match_errors-to_reply-to,domain_match_sender_from,domain_match_references_reply-to,domain_match_references_in-reply-to,domain_match_references_to,domain_match_from_reply-to,domain_match_to_from,domain_match_to_message-id,domain_match_to_received,label
0,1,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,1,1,0,1
1,1,0,0,0,0,1,1,1,1,1,...,0,0,0,1,0,0,1,0,0,1
2,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,1,0,0,1
3,1,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
4,2,0,0,0,0,1,0,1,1,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2572,1,0,0,0,0,1,0,1,1,0,...,0,0,0,1,0,0,0,0,0,0
2573,2,0,0,0,0,1,0,1,1,0,...,0,0,0,1,1,0,1,1,0,0
2574,2,0,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


**Reduce feature set:**

The only features that are kept are domain matching features, as these should generalize across very different email datasets without issue.

In [11]:
feature_list = [
'domain_match_from_return-path',
'domain_match_message-id_from',
'domain_match_message-id_return-path',
'domain_match_to_from',
'domain_match_errors-to_from',
'domain_match_message-id_reply-to',
'domain_match_errors-to_message-id',
'domain_match_sender_from',
'domain_match_to_received',
'domain_match_errors-to_reply-to',
'domain_match_to_message-id',
'label']

feature_list = ['domain_val_message-id',
       'domain_match_message-id_from', 'domain_match_from_return-path',
       'domain_match_message-id_return-path', 'domain_match_message-id_sender',
       'domain_match_message-id_reply-to', 'domain_match_return-path_reply-to',
       'domain_match_reply-to_to', 'domain_match_to_in-reply-to',
       'domain_match_errors-to_message-id', 'domain_match_errors-to_from',
       'domain_match_errors-to_sender', 'domain_match_errors-to_reply-to',
       'domain_match_sender_from', 'domain_match_references_reply-to',
       'domain_match_references_in-reply-to', 'domain_match_references_to',
       'domain_match_from_reply-to', 'domain_match_to_from',
       'domain_match_to_message-id', 'domain_match_to_received', 'label']

df = df[feature_list]

In [12]:
df_Y = df['label']
df_X = df.drop('label', axis=1)

In [13]:
df_X.shape

(2576, 21)

In [14]:
features_list = df_X.columns

**Apply a standard scaler to the full data set:**

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(df_X)
df_X = scaler.transform(df_X)
df_X = pd.DataFrame(df_X, columns=features_list)

**Breaking the data into a test and training set (20% test, 80% train)**

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.20, random_state=42)

In [17]:
X_train.shape

(2060, 21)

In [18]:
X_test.shape

(516, 21)

# **Hyperparameter Tuning and Testing:**

**Random Forest:**

In [19]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("rf", RandomForestClassifier())
                ])

param_grid_list = {'rf__n_estimators': [100, 150],
                  'rf__criterion': ['entropy', 'gini'],
                  'rf__min_samples_split': [2, 3],
                  'rf__min_samples_leaf': [1, 2],
                  'rf__max_features': ['auto', 'sqrt', 'log2']}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
rf_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

rf_df[rf_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


{'rf__criterion': 'entropy', 'rf__max_features': 'auto', 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 150} 

Pipeline(steps=[('scale', StandardScaler()),
                ('rf',
                 RandomForestClassifier(criterion='entropy',
                                        max_features='auto',
                                        n_estimators=150))])


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Accuracy: 97.48062015503875
F1 Score: 97.44597249508841
Recall: 100.0
Precision: 95.01915708812261
ROC AUC: 97.57462686567165
Confusion Matrix: [[255  13]
 [  0 248]]
CPU times: user 3.05 s, sys: 419 ms, total: 3.47 s
Wall time: 1min 58s


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rf__criterion,param_rf__max_features,param_rf__min_samples_leaf,param_rf__min_samples_split,param_rf__n_estimators,params,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
1,0.486675,0.015811,0.022342,0.001050,entropy,auto,1,2,150,"{'rf__criterion': 'entropy', 'rf__max_features...",...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.980583,0.981068,0.007346,1
3,0.482647,0.008781,0.021599,0.000760,entropy,auto,1,3,150,"{'rf__criterion': 'entropy', 'rf__max_features...",...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.980583,0.980583,0.007520,4
17,0.472300,0.008128,0.023178,0.004299,entropy,log2,1,2,150,"{'rf__criterion': 'entropy', 'rf__max_features...",...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.980583,0.981068,0.007346,1
19,0.467691,0.009346,0.021126,0.000792,entropy,log2,1,3,150,"{'rf__criterion': 'entropy', 'rf__max_features...",...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.980583,0.980583,0.007520,4
24,0.325446,0.008834,0.018405,0.005731,gini,auto,1,2,100,"{'rf__criterion': 'gini', 'rf__max_features': ...",...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.980583,0.981068,0.007346,1


**MLP:**

In [28]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("mlp", MLPClassifier())
                ])

param_grid_list = {'mlp__hidden_layer_sizes': [(20,), (20,20), (40,), (40,40)],
                   'mlp__activation': ['tanh', 'relu'],
                   'mlp__learning_rate': ['constant', 'adaptive'],
                   'mlp__solver': ['adam', 'sgd'],
                   'mlp__alpha': [0.0001, 0.001, 0.01]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
mlp_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

mlp_df[mlp_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 96 candidates, totalling 960 fits
{'mlp__activation': 'tanh', 'mlp__alpha': 0.0001, 'mlp__hidden_layer_sizes': (20,), 'mlp__learning_rate': 'constant', 'mlp__solver': 'adam'} 

Pipeline(steps=[('scale', StandardScaler()),
                ('mlp',
                 MLPClassifier(activation='tanh', hidden_layer_sizes=(20,)))])
Accuracy: 97.48062015503875
F1 Score: 97.44597249508841
Recall: 100.0
Precision: 95.01915708812261
ROC AUC: 97.57462686567165
Confusion Matrix: [[255  13]
 [  0 248]]
CPU times: user 16 s, sys: 2.68 s, total: 18.7 s
Wall time: 18min 13s


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_mlp__activation,param_mlp__alpha,param_mlp__hidden_layer_sizes,param_mlp__learning_rate,param_mlp__solver,params,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,2.017966,0.612361,0.005223,0.002303,tanh,0.0001,"(20,)",constant,adam,"{'mlp__activation': 'tanh', 'mlp__alpha': 0.00...",...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.982524,0.007583,1
8,2.487094,0.657803,0.005717,0.003265,tanh,0.0001,"(40,)",constant,adam,"{'mlp__activation': 'tanh', 'mlp__alpha': 0.00...",...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.982524,0.007583,1
10,2.340940,0.636701,0.005299,0.001890,tanh,0.0001,"(40,)",adaptive,adam,"{'mlp__activation': 'tanh', 'mlp__alpha': 0.00...",...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.982524,0.007583,1
30,2.569576,0.525426,0.005962,0.001807,tanh,0.001,"(40, 40)",adaptive,adam,"{'mlp__activation': 'tanh', 'mlp__alpha': 0.00...",...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.982524,0.007583,1
52,1.538848,0.470811,0.005928,0.003419,relu,0.0001,"(20, 20)",constant,adam,"{'mlp__activation': 'relu', 'mlp__alpha': 0.00...",...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.982524,0.007583,1


**Logistic Regression:**

In [21]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("lr", LogisticRegression())
                ])

param_grid_list = {'lr__max_iter': [500],
                  'lr__solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
                  'lr__fit_intercept': [True, False],
                  'lr__tol': [0.0001, 0.001],
                  'lr__penalty': ['l1', 'l2', 'elasticnet'],
                  'lr__C': [0.1, 1, 10]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
lr_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

lr_df[lr_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
840 fits failed out of a total of 1440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dua

{'lr__C': 1, 'lr__fit_intercept': True, 'lr__max_iter': 500, 'lr__penalty': 'l1', 'lr__solver': 'saga', 'lr__tol': 0.001} 

Pipeline(steps=[('scale', StandardScaler()),
                ('lr',
                 LogisticRegression(C=1, max_iter=500, penalty='l1',
                                    solver='saga', tol=0.001))])
Accuracy: 97.28682170542635
F1 Score: 97.23320158102766
Recall: 99.19354838709677
Precision: 95.34883720930233
ROC AUC: 97.35796822339914
Confusion Matrix: [[256  12]
 [  2 246]]
CPU times: user 1.84 s, sys: 164 ms, total: 2.01 s
Wall time: 43.3 s


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lr__C,param_lr__fit_intercept,param_lr__max_iter,param_lr__penalty,param_lr__solver,param_lr__tol,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
55,0.140613,0.006364,0.003913,0.000868,1,True,500,l1,saga,0.001,...,0.966019,0.966019,0.985437,0.980583,0.985437,0.966019,0.985437,0.978155,0.008751,1
56,0.028316,0.001482,0.003455,0.000150,1,True,500,l2,newton-cg,0.0001,...,0.966019,0.966019,0.985437,0.980583,0.985437,0.966019,0.985437,0.978155,0.008751,1
57,0.027199,0.001971,0.003459,0.000196,1,True,500,l2,newton-cg,0.001,...,0.966019,0.966019,0.985437,0.980583,0.985437,0.966019,0.985437,0.978155,0.008751,1
58,0.023103,0.003573,0.003419,0.000102,1,True,500,l2,lbfgs,0.0001,...,0.966019,0.966019,0.985437,0.980583,0.985437,0.966019,0.985437,0.978155,0.008751,1
59,0.022022,0.001098,0.003435,0.000130,1,True,500,l2,lbfgs,0.001,...,0.966019,0.966019,0.985437,0.980583,0.985437,0.966019,0.985437,0.978155,0.008751,1


**SVM:**

In [22]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("svc", SVC())
                ])

param_grid_list = {'svc__C': [0.1, 1, 10],
                  'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                  'svc__degree': [3, 4, 5],
                  'svc__tol': [0.001, 0.0001, 0.01]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
svm_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

svm_df[svm_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits
{'svc__C': 10, 'svc__degree': 3, 'svc__kernel': 'rbf', 'svc__tol': 0.001} 

Pipeline(steps=[('scale', StandardScaler()), ('svc', SVC(C=10))])
Accuracy: 97.48062015503875
F1 Score: 97.44597249508841
Recall: 100.0
Precision: 95.01915708812261
ROC AUC: 97.57462686567165
Confusion Matrix: [[255  13]
 [  0 248]]
CPU times: user 1.89 s, sys: 195 ms, total: 2.09 s
Wall time: 52 s


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svc__C,param_svc__degree,param_svc__kernel,param_svc__tol,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
78,0.033201,0.001317,0.007554,0.000254,10,3,rbf,0.001,"{'svc__C': 10, 'svc__degree': 3, 'svc__kernel'...",0.975728,...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.980583,0.007827,1
79,0.035800,0.002834,0.007826,0.000793,10,3,rbf,0.0001,"{'svc__C': 10, 'svc__degree': 3, 'svc__kernel'...",0.975728,...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.980583,0.007827,1
80,0.034180,0.006253,0.007581,0.000706,10,3,rbf,0.01,"{'svc__C': 10, 'svc__degree': 3, 'svc__kernel'...",0.975728,...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.980583,0.007827,1
90,0.032861,0.001303,0.007055,0.000322,10,4,rbf,0.001,"{'svc__C': 10, 'svc__degree': 4, 'svc__kernel'...",0.975728,...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.980583,0.007827,1
91,0.035793,0.003362,0.007612,0.000524,10,4,rbf,0.0001,"{'svc__C': 10, 'svc__degree': 4, 'svc__kernel'...",0.975728,...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.980583,0.007827,1


**Decision Tree:**

In [23]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("dt", DecisionTreeClassifier())
                ])

param_grid_list = {'dt__criterion': ['entropy', 'gini'],
                  'dt__min_samples_split': [2, 3, 4],
                  'dt__min_samples_leaf': [1, 2, 3],
                  'dt__ccp_alpha': [0, 0.005, 0.01, 0.025, 0.05, 0.1]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
dt_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

dt_df[dt_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits
{'dt__ccp_alpha': 0, 'dt__criterion': 'entropy', 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 2} 

Pipeline(steps=[('scale', StandardScaler()),
                ('dt',
                 DecisionTreeClassifier(ccp_alpha=0, criterion='entropy'))])
Accuracy: 97.48062015503875
F1 Score: 97.44597249508841
Recall: 100.0
Precision: 95.01915708812261
ROC AUC: 97.57462686567165
Confusion Matrix: [[255  13]
 [  0 248]]
CPU times: user 980 ms, sys: 38.5 ms, total: 1.02 s
Wall time: 9.69 s


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_dt__ccp_alpha,param_dt__criterion,param_dt__min_samples_leaf,param_dt__min_samples_split,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010912,0.002618,0.003623,0.000691,0,entropy,1,2,"{'dt__ccp_alpha': 0, 'dt__criterion': 'entropy...",0.970874,...,0.966019,0.985437,0.985437,0.980583,0.985437,0.970874,0.980583,0.978641,0.007887,1
1,0.010489,0.002436,0.003475,0.000193,0,entropy,1,3,"{'dt__ccp_alpha': 0, 'dt__criterion': 'entropy...",0.970874,...,0.966019,0.985437,0.985437,0.980583,0.985437,0.970874,0.980583,0.978641,0.007887,1
2,0.012077,0.003843,0.005016,0.004255,0,entropy,1,4,"{'dt__ccp_alpha': 0, 'dt__criterion': 'entropy...",0.970874,...,0.966019,0.985437,0.985437,0.980583,0.985437,0.970874,0.980583,0.978641,0.007887,1
9,0.013053,0.005212,0.003472,0.000204,0,gini,1,2,"{'dt__ccp_alpha': 0, 'dt__criterion': 'gini', ...",0.970874,...,0.961165,0.990291,0.985437,0.980583,0.985437,0.970874,0.980583,0.978641,0.009262,1
10,0.009305,0.000722,0.003699,0.000841,0,gini,1,3,"{'dt__ccp_alpha': 0, 'dt__criterion': 'gini', ...",0.970874,...,0.961165,0.985437,0.985437,0.980583,0.985437,0.970874,0.980583,0.978155,0.008751,5


**Naive Bayes (Gaussian):**

In [24]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("gnb", GaussianNB())
                ])

param_grid_list = {'gnb__var_smoothing': [1E-9, 1E-10, 1E-8]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
nb_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

nb_df[nb_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 3 candidates, totalling 30 fits
{'gnb__var_smoothing': 1e-10} 

Pipeline(steps=[('scale', StandardScaler()),
                ('gnb', GaussianNB(var_smoothing=1e-10))])
Accuracy: 86.62790697674419
F1 Score: 87.74422735346359
Recall: 99.59677419354838
Precision: 78.41269841269842
ROC AUC: 87.1118199325951
Confusion Matrix: [[200  68]
 [  1 247]]
CPU times: user 80.3 ms, sys: 6.65 ms, total: 87 ms
Wall time: 312 ms


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gnb__var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009838,0.001690,0.003714,0.000148,0.0,{'gnb__var_smoothing': 1e-09},0.902913,0.878641,0.902913,0.883495,0.902913,0.893204,0.88835,0.907767,0.912621,0.912621,0.898544,0.011374,3
1,0.012230,0.006958,0.003653,0.000147,0.0,{'gnb__var_smoothing': 1e-10},0.902913,0.878641,0.902913,0.883495,0.902913,0.893204,0.88835,0.927184,0.912621,0.912621,0.900485,0.014111,1
2,0.009169,0.002220,0.003767,0.000995,0.0,{'gnb__var_smoothing': 1e-08},0.902913,0.878641,0.907767,0.883495,0.902913,0.893204,0.88835,0.917476,0.912621,0.912621,0.900000,0.012696,2


**AdaBoost:**

In [25]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("ab", AdaBoostClassifier())
                ])

param_grid_list = {'ab__n_estimators': [50, 100, 150, 200],
                  'ab__learning_rate': [0.95, 1, 1.05, 1.25, 1.5, 1.75, 2],
                  'ab__algorithm': ['SAMME', 'SAMME.R']}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
ab_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

ab_df[ab_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 56 candidates, totalling 560 fits
{'ab__algorithm': 'SAMME', 'ab__learning_rate': 0.95, 'ab__n_estimators': 200} 

Pipeline(steps=[('scale', StandardScaler()),
                ('ab',
                 AdaBoostClassifier(algorithm='SAMME', learning_rate=0.95,
                                    n_estimators=200))])
Accuracy: 96.89922480620154
F1 Score: 96.85039370078741
Recall: 99.19354838709677
Precision: 94.61538461538461
ROC AUC: 96.98483389504092
Confusion Matrix: [[254  14]
 [  2 246]]
CPU times: user 4.58 s, sys: 458 ms, total: 5.04 s
Wall time: 2min 51s


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ab__algorithm,param_ab__learning_rate,param_ab__n_estimators,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
2,0.543378,0.015919,0.037714,0.002839,SAMME,0.95,150,"{'ab__algorithm': 'SAMME', 'ab__learning_rate'...",0.975728,0.966019,...,0.961165,0.966019,0.980583,0.980583,0.985437,0.966019,0.985437,0.975728,0.009709,5
3,1.019286,0.342097,0.053524,0.012433,SAMME,0.95,200,"{'ab__algorithm': 'SAMME', 'ab__learning_rate'...",0.975728,0.966019,...,0.961165,0.970874,0.980583,0.980583,0.985437,0.966019,0.985437,0.976214,0.009325,1
7,0.806138,0.182064,0.058617,0.012830,SAMME,1,200,"{'ab__algorithm': 'SAMME', 'ab__learning_rate'...",0.975728,0.966019,...,0.961165,0.970874,0.980583,0.980583,0.985437,0.966019,0.985437,0.976214,0.009325,1
10,0.543109,0.013679,0.036970,0.001812,SAMME,1.05,150,"{'ab__algorithm': 'SAMME', 'ab__learning_rate'...",0.975728,0.966019,...,0.961165,0.970874,0.980583,0.980583,0.985437,0.966019,0.985437,0.976214,0.009325,1
11,0.713010,0.013062,0.047558,0.003182,SAMME,1.05,200,"{'ab__algorithm': 'SAMME', 'ab__learning_rate'...",0.975728,0.966019,...,0.961165,0.970874,0.980583,0.980583,0.985437,0.966019,0.985437,0.976214,0.009325,1


**GradientBoostingClassifier:**

In [26]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("gbc", GradientBoostingClassifier())
                ])

param_grid_list = {'gbc__max_features': ['auto', 'sqrt', 'log2'],
                   'gbc__learning_rate': [0.05, 0.1, 0.2, 0.25, 0.30, 0.35, 0.40, 0.5, 0.6, 0.7, 0.9],
                   'gbc__n_estimators': [100, 200]
                  }

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
gb_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

gb_df[gb_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 66 candidates, totalling 660 fits
{'gbc__learning_rate': 0.1, 'gbc__max_features': 'log2', 'gbc__n_estimators': 200} 

Pipeline(steps=[('scale', StandardScaler()),
                ('gbc',
                 GradientBoostingClassifier(max_features='log2',
                                            n_estimators=200))])
Accuracy: 97.48062015503875
F1 Score: 97.44597249508841
Recall: 100.0
Precision: 95.01915708812261
ROC AUC: 97.57462686567165
Confusion Matrix: [[255  13]
 [  0 248]]
CPU times: user 4.01 s, sys: 478 ms, total: 4.49 s
Wall time: 2min 50s


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gbc__learning_rate,param_gbc__max_features,param_gbc__n_estimators,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
11,0.422462,0.013656,0.005656,0.000448,0.1,log2,200,"{'gbc__learning_rate': 0.1, 'gbc__max_features...",0.985437,0.985437,...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.982524,0.007583,1
28,0.207990,0.008145,0.004814,0.000109,0.3,log2,100,"{'gbc__learning_rate': 0.3, 'gbc__max_features...",0.985437,0.985437,...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.982524,0.007583,1
34,0.211547,0.010016,0.004911,0.000177,0.35,log2,100,"{'gbc__learning_rate': 0.35, 'gbc__max_feature...",0.985437,0.985437,...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.982524,0.007583,1
50,0.379725,0.062240,0.005189,0.001676,0.6,sqrt,100,"{'gbc__learning_rate': 0.6, 'gbc__max_features...",0.985437,0.985437,...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.982524,0.007583,1
56,0.397224,0.048486,0.007862,0.004206,0.7,sqrt,100,"{'gbc__learning_rate': 0.7, 'gbc__max_features...",0.985437,0.985437,...,0.966019,0.990291,0.985437,0.980583,0.985437,0.970874,0.985437,0.982524,0.007583,1


**KNN:**

In [27]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("knn", KNeighborsClassifier())
                ])

param_grid_list = {'knn__n_neighbors': [1, 10, 20],
                  'knn__weights': ['uniform', 'distance'],
                  'knn__p': [1, 2],
                  'knn__algorithm': ['auto'],
                  'knn__leaf_size': [15, 30, 45]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
knn_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

knn_df[knn_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 36 candidates, totalling 360 fits
{'knn__algorithm': 'auto', 'knn__leaf_size': 15, 'knn__n_neighbors': 10, 'knn__p': 1, 'knn__weights': 'distance'} 

Pipeline(steps=[('scale', StandardScaler()),
                ('knn',
                 KNeighborsClassifier(leaf_size=15, n_neighbors=10, p=1,
                                      weights='distance'))])
Accuracy: 97.28682170542635
F1 Score: 97.25490196078431
Recall: 100.0
Precision: 94.65648854961832
ROC AUC: 97.38805970149254
Confusion Matrix: [[254  14]
 [  0 248]]
CPU times: user 637 ms, sys: 70.4 ms, total: 707 ms
Wall time: 8.82 s


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_knn__algorithm,param_knn__leaf_size,param_knn__n_neighbors,param_knn__p,param_knn__weights,params,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
5,0.008060,0.001626,0.027856,0.002940,auto,15,10,1,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 1...",...,0.966019,0.990291,0.985437,0.980583,0.980583,0.970874,0.985437,0.981068,0.007660,1
11,0.008483,0.002080,0.010452,0.001501,auto,15,20,2,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 1...",...,0.966019,0.990291,0.985437,0.980583,0.980583,0.966019,0.985437,0.980583,0.008408,4
17,0.008097,0.001443,0.026983,0.002892,auto,30,10,1,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 3...",...,0.966019,0.990291,0.985437,0.980583,0.980583,0.970874,0.985437,0.981068,0.007660,1
23,0.013575,0.005703,0.017178,0.005897,auto,30,20,2,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 3...",...,0.966019,0.990291,0.985437,0.980583,0.980583,0.966019,0.985437,0.980583,0.008408,4
29,0.009007,0.003730,0.026388,0.001353,auto,45,10,1,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 4...",...,0.966019,0.990291,0.985437,0.980583,0.980583,0.970874,0.985437,0.981068,0.007660,1


# **Stacked Testing:**

In [29]:
from sklearn.ensemble import StackingClassifier

'''
base_learners = [('rf', RandomForestClassifier(criterion='entropy', max_features='auto', min_samples_leaf=1, min_samples_split=3, n_estimators=100)),
                ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')),
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]
'''

base_learners_set1 = [('rf', RandomForestClassifier(criterion='entropy', max_features='auto', min_samples_leaf=1, min_samples_split=3, n_estimators=100)),
                ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance'))]

base_learners_set2 = [('rf', RandomForestClassifier(criterion='entropy', max_features='auto', min_samples_leaf=1, min_samples_split=3, n_estimators=100)),
                ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]

base_learners_set3 = [('rf', RandomForestClassifier(criterion='entropy', max_features='auto', min_samples_leaf=1, min_samples_split=3, n_estimators=100)),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')),
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]

base_learners_set4 = [('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')),
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]

base_learners = []
base_learners.append(base_learners_set1)
base_learners.append(base_learners_set2)
base_learners.append(base_learners_set3)
base_learners.append(base_learners_set4)

for base_learner_group in base_learners:

    meta_learner = LogisticRegression()

    clf = StackingClassifier(estimators=base_learner_group, final_estimator=meta_learner)

    # Train the stacked model on the full training data
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)

    # Get the evaluation metrics
    print('Accuracy:', accuracy_score(y_test, predictions)*100)
    print('F1 Score:', f1_score(y_test, predictions)*100)
    print('Recall:', recall_score(y_test, predictions)*100)
    print('Precision:', precision_score(y_test, predictions)*100)
    print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
    print('Confusion Matrix:', confusion_matrix(y_test, predictions))
    print('-----------------------------------------\n')

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

Accuracy: 97.48062015503875
F1 Score: 97.44597249508841
Recall: 100.0
Precision: 95.01915708812261
ROC AUC: 97.57462686567165
Confusion Matrix: [[255  13]
 [  0 248]]
-----------------------------------------



/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

Accuracy: 97.48062015503875
F1 Score: 97.44597249508841
Recall: 100.0
Precision: 95.01915708812261
ROC AUC: 97.57462686567165
Confusion Matrix: [[255  13]
 [  0 248]]
-----------------------------------------



/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

Accuracy: 97.48062015503875
F1 Score: 97.44597249508841
Recall: 100.0
Precision: 95.01915708812261
ROC AUC: 97.57462686567165
Confusion Matrix: [[255  13]
 [  0 248]]
-----------------------------------------

Accuracy: 97.48062015503875
F1 Score: 97.44597249508841
Recall: 100.0
Precision: 95.01915708812261
ROC AUC: 97.57462686567165
Confusion Matrix: [[255  13]
 [  0 248]]
-----------------------------------------

